# Weights Version Control

In [1]:
from grid.ipfsapi.client import Client
import numpy as np
import pickle
client = Client()

/Users/yanndupis/anaconda2/envs/openmined2/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


This notebook illustrate version control of the weights with VersionTreeNode api.
- First we split the MNIST data in three parts.
- We train the model on each part. Every time we train the model on a subset of the original dataset, we commit the weights.
- VersionTreeNode gives us the abilty to get access to the historical version of the weights by listing ancestors.

In [2]:
# TO DELETE once it's merged to grid master bran
""" This module contains an implementation of an IPFS
version-control system, which is structured as a directed in-tree with nodes
represented by the bytes representation of the VersionTreeNode class. """
from typing import Optional, Iterator

from grid import ipfsapi

# TODO: Unit tests.
# TODO: Do we want to store the hash on the node after it's been committed?
class VersionTreeNode:
    """ Thin wrapper around a piece of IPFS-versioned data and the
    IPFS multihash of its parent. """
    # Delimiter for serializing packed object.
    DELIMITER = b"|"

    def __init__(self,
                 contents: bytes,
                 parent_hash: Optional[str] = None,
                 ipfs_client: ipfsapi.Client = None):
        """ parent_hash is a UTF-8 IPFS multihash identifying
        this node's parent in the version tree. If parent_hash is None,
        this node is the root of a version tree. """
        self.contents = contents
        # Convert empty string to None to minimize typing bugs.
        # Porbably should be encode() but won't be able to encode None
        #self.parent_hash = parent_hash.decode("utf-8") or None
        self.parent_hash = parent_hash or None
        self.ipfs_client = ipfs_client

    # TODO: Should there just be a singleton IPFS client in the ipfsapi.client
    # module?  It seems like the current architecture would require
    # it to either be passed around all over the place or potentially go
    # through the connection process multiple times, and I don't think
    # multiple configurations need to be maintained for a given worker node.
    def commit(self, ipfs_client: ipfsapi.Client = None) -> str:
        """ Commits the node to the version tree, and returns the
        UTF-8 multihash representing its IPFS ID"""
        return (ipfs_client or self.ipfs_client).add_bytes(self.to_bytes())

    @staticmethod
    def get_node_by_hash(multihash: str,
                         ipfs_client: ipfsapi.Client) -> "VersionTreeNode":
        """ Retrieve and deserialize a VersionTreeNode addressed
        by it's UTF-8 multihash IPFS ID. """
        return VersionTreeNode.from_bytes(ipfs_client.cat(multihash))

    def get_with_ancestors(
            self,
            ipfs_client: ipfsapi.Client = None) -> Iterator["VersionTreeNode"]:
        """ Return an iterator containing this node and all its
        direct ancestors in the version tree, in that order. """
        yield self
        parent_hash = self.parent_hash
        while parent_hash is not None:
            parent_node = VersionTreeNode.get_node_by_hash(
                parent_hash,
                (ipfs_client or self.ipfs_client))
            parent_hash = parent_node.parent_hash
            yield parent_node

    @staticmethod
    def get_node_with_ancestors_by_hash(
            multihash: str,
            ipfs_client: ipfsapi.Client) -> Iterator["VersionTreeNode"]:
        """ Convenience method to get an iterator of the node identified by the
        provided UTF-8 IPFS multihash, along with all of its ancestors, in
        that order."""
        return VersionTreeNode.get_node_by_hash(
            multihash, ipfs_client).get_with_ancestors(ipfs_client)

    def to_bytes(self) -> bytes:
        """ For contents b"foo", parent_hash "bar", and DELIMITER b"|",
        returns b"foo|bar" """
        parent_hash_bytes = self.parent_hash.encode("utf-8") if \
            self.parent_hash else \
            b""
        return VersionTreeNode.DELIMITER.join(
            (self.contents, parent_hash_bytes)
        )

    @staticmethod
    def from_bytes(b: bytes) -> "VersionTreeNode":
        return VersionTreeNode(*b.split(VersionTreeNode.DELIMITER))

Load MNIST data.

In [3]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Split the data.

In [4]:
size_part = len(x_train)//3
x_train_part_1 = x_train[:size_part,: :]
y_train_part_1 = y_train[:size_part]

x_train_part_2 = x_train[size_part:size_part*2,: :] 
y_train_part_2 = y_train[size_part:size_part*2]

x_train_part_3 = x_train[size_part*2:size_part*3,: :]
y_train_part_3 =  y_train[size_part*2:size_part*3]

Create the model.

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

#### Fit model on the first part then commit weights

In [6]:
model.fit(x_train_part_1, y_train_part_1,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 69s 3ms/step - loss: 0.5215 - acc: 0.8368 - val_loss: 0.1596 - val_acc: 0.9466
Test loss: 0.15963942354917526
Test accuracy: 0.9466


Commit weigths after training the model on the first part of the MNIST data.

In [7]:
# Get the weights of the model
weights_part_1 = model.get_weights()
# Create a node to store the weights
n1 = VersionTreeNode(pickle.dumps(weights_part_1), None)
# commit the node to write the weights to IPFS. 
# Set parent_hash to None if the first version of weights (root of the tree)
n1_ipfs_address = n1.commit(client)
print(n1_ipfs_address)

model.set_weights(pickle.loads(n1.contents))

Qmd5dAkM79xewwHobhvkcSwTuX6sKRYukDx2AzXcYkAGGN


#### Fit model on the second part then commit weights

In [8]:
model.fit(x_train_part_2, y_train_part_2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 74s 4ms/step - loss: 0.1650 - acc: 0.9507 - val_loss: 0.0792 - val_acc: 0.9760
Test loss: 0.07918845291715115
Test accuracy: 0.976


In [9]:
weights_part_2 = model.get_weights()

n2 = VersionTreeNode(pickle.dumps(weights_part_2), n1_ipfs_address)

n2_ipfs_address = n2.commit(client)
print(n2_ipfs_address)

model.set_weights(pickle.loads(n2.contents))

QmeHuL2NeFDrcX3uiWoHsCexuEna3eCHfT7rkxAXyB3xuZ


#### Fit model on the third part then commit weights

In [10]:
model.fit(x_train_part_3, y_train_part_3,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 20000 samples, validate on 10000 samples
Epoch 1/1
20000/20000 [==============================] - 70s 3ms/step - loss: 0.1203 - acc: 0.9643 - val_loss: 0.0559 - val_acc: 0.9816
Test loss: 0.055859832402365285
Test accuracy: 0.9816


In [11]:
weights_part_3 = model.get_weights()

n3 = VersionTreeNode(pickle.dumps(weights_part_3), parent_hash=n2_ipfs_address)

n3_ipfs_address = n3.commit(client)

#### Get historical weights

In [12]:
node_ancestors = n3.get_with_ancestors(client)

In [13]:
historical_weights = [pickle.loads(x.contents) for x in node_ancestors]

TypeError: __init__() takes from 2 to 4 positional arguments but 13114 were given

In [ ]:
for i in range(len(historical_weights)):
    model.set_weights(weights_log[i])
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])